## CASO DE NEGOCIO 2
Utiliza Python a través de Jupyter Notebook

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark= SparkSession.builder.appName('firstSession')\
       .config('spark.master','local[4]')\
       .config('spark.shuffle.sql.partitions','1')\
       .getOrCreate()   
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

dtf_1 = spark.read.csv('vehicles.csv', sep=',', header=True)

In [2]:
dtf_1.count()

441802

In [5]:
dtf_1.printSchema()

root
 |-- id: string (nullable = true)
 |-- url: string (nullable = true)
 |-- region: string (nullable = true)
 |-- region_url: string (nullable = true)
 |-- price: string (nullable = true)
 |-- year: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- condition: string (nullable = true)
 |-- cylinders: string (nullable = true)
 |-- fuel: string (nullable = true)
 |-- odometer: string (nullable = true)
 |-- title_status: string (nullable = true)
 |-- transmission: string (nullable = true)
 |-- VIN: string (nullable = true)
 |-- drive: string (nullable = true)
 |-- size: string (nullable = true)
 |-- type: string (nullable = true)
 |-- paint_color: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- description: string (nullable = true)
 |-- county: string (nullable = true)
 |-- state: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- long: string (nullable = true)
 |-- posting_date: string (nu

## 1. En el campo year deja solo el año, quitando el “.” Y el “0”

In [36]:
# hay valores nulos y no se encuentra el . se trabaja dejando los dos ultimos digitos
from pyspark.sql.functions import col, substring
dtf_1.select(substring(dtf_1.year, 2, 2).alias('s')).collect()

[Row(s=None),
 Row(s=None),
 Row(s=None),
 Row(s=None),
 Row(s=None),
 Row(s=None),
 Row(s=None),
 Row(s=None),
 Row(s=None),
 Row(s=None),
 Row(s=None),
 Row(s=None),
 Row(s=None),
 Row(s=None),
 Row(s=None),
 Row(s=None),
 Row(s=None),
 Row(s=None),
 Row(s=None),
 Row(s=None),
 Row(s=None),
 Row(s=None),
 Row(s=None),
 Row(s=None),
 Row(s=None),
 Row(s=None),
 Row(s=None),
 Row(s='01'),
 Row(s='01'),
 Row(s='02'),
 Row(s='01'),
 Row(s='01'),
 Row(s='01'),
 Row(s='01'),
 Row(s='01'),
 Row(s='01'),
 Row(s='01'),
 Row(s='99'),
 Row(s='01'),
 Row(s='01'),
 Row(s='01'),
 Row(s='01'),
 Row(s='01'),
 Row(s='01'),
 Row(s='01'),
 Row(s='01'),
 Row(s='01'),
 Row(s='01'),
 Row(s='01'),
 Row(s='02'),
 Row(s='02'),
 Row(s='01'),
 Row(s='02'),
 Row(s='02'),
 Row(s='01'),
 Row(s='00'),
 Row(s='01'),
 Row(s='01'),
 Row(s='01'),
 Row(s='01'),
 Row(s='01'),
 Row(s='01'),
 Row(s='01'),
 Row(s='01'),
 Row(s='01'),
 Row(s='00'),
 Row(s='01'),
 Row(s='01'),
 Row(s='01'),
 Row(s='01'),
 Row(s='01'),
 Row(s

## 2. Informa cuantas regiones diferentes hay y cuantos registros hay por cada región

In [7]:
from pyspark.sql.functions import countDistinct
gr = dtf_1.groupBy("region").agg(countDistinct("id"))
gr.show()


+--------------------+---------+
|              region|count(id)|
+--------------------+---------+
|          western KY|      212|
|   central louisiana|      132|
|           texarkana|      211|
| do not hesitate ...|        1|
|       daytona beach|     1633|
|   heartland florida|      170|
|  southeast missouri|      245|
|            sandusky|      244|
|        gold country|      765|
|           pensacola|      622|
|                  ct|       93|
|        south jersey|     2974|
|       new hampshire|     2981|
|          san marcos|      179|
|           fairbanks|      427|
|     kansas city, MO|     2957|
|          harrisburg|      976|
|       state college|      210|
|              yakima|     1704|
| do not hesitate ...|        1|
+--------------------+---------+
only showing top 20 rows



## Genera un archivo con los registros que están completos descartando aquellos que tienen algunos de sus campos nulos

In [18]:
dtf_1.na.drop("all").show(truncate=False)

+----------+-----------------------------------------------------------------------------------------------+----------------------+-----------------------------------+-----+----+------------+-----+---------+---------+----+--------+------------+------------+----+-----+----+----+-----------+---------+-----------+------+-----+----+----+------------+
|id        |url                                                                                            |region                |region_url                         |price|year|manufacturer|model|condition|cylinders|fuel|odometer|title_status|transmission|VIN |drive|size|type|paint_color|image_url|description|county|state|lat |long|posting_date|
+----------+-----------------------------------------------------------------------------------------------+----------------------+-----------------------------------+-----+----+------------+-----+---------+---------+----+--------+------------+------------+----+-----+----+----+-----------+---------+--

In [19]:
dtf_1.count()

441802

## Trae los registros donde el price sea >= 15000 y la condición del auto sea good o excellent.

In [24]:
dtf_1.filter((dtf_1.price >= 15000) & (dtf_1.condition == 'GOOD') | (dtf_1.condition == 'EXCELLENT')).show(truncate=False)

+---+---+------+----------+-----+----+------------+-----+---------+---------+----+--------+------------+------------+---+-----+----+----+-----------+---------+-----------+------+-----+---+----+------------+
|id |url|region|region_url|price|year|manufacturer|model|condition|cylinders|fuel|odometer|title_status|transmission|VIN|drive|size|type|paint_color|image_url|description|county|state|lat|long|posting_date|
+---+---+------+----------+-----+----+------------+-----+---------+---------+----+--------+------------+------------+---+-----+----+----+-----------+---------+-----------+------+-----+---+----+------------+
+---+---+------+----------+-----+----+------------+-----+---------+---------+----+--------+------------+------------+---+-----+----+----+-----------+---------+-----------+------+-----+---+----+------------+



## Realiza el conteo de los autos azules manufacturados por chevrolet

In [39]:
from pyspark.sql.functions import countDistinct
gr1 = dtf_1.groupBy("paint_color").agg(countDistinct((dtf_1.paint_color=='BLUE') & (dtf_1.manufacturer=='CHEVROLET')))
gr1.show()

Py4JJavaError: An error occurred while calling o158.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 39.0 failed 1 times, most recent failure: Lost task 0.0 in stage 39.0 (TID 123) (DESKTOP-8RH83RD.mshome.net executor driver): java.io.FileNotFoundException: C:\Users\ASUS\AppData\Local\Temp\blockmgr-886070c8-7742-4866-b010-48db936c2eb0\3c\temp_shuffle_b90d8309-5605-4310-a93f-2ae48dcee121 (The system cannot find the path specified)
	at java.io.FileOutputStream.open0(Native Method)
	at java.io.FileOutputStream.open(FileOutputStream.java:270)
	at java.io.FileOutputStream.<init>(FileOutputStream.java:213)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:140)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:159)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:306)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.

In [41]:
dtf_1.select('manufacturer').distinct().collect()

Py4JJavaError: An error occurred while calling o172.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 41.0 failed 1 times, most recent failure: Lost task 0.0 in stage 41.0 (TID 135) (DESKTOP-8RH83RD.mshome.net executor driver): java.io.FileNotFoundException: C:\Users\ASUS\AppData\Local\Temp\blockmgr-886070c8-7742-4866-b010-48db936c2eb0\0f\temp_shuffle_6a1fd301-eeff-4803-95f3-20744632e559 (The system cannot find the path specified)
	at java.io.FileOutputStream.open0(Native Method)
	at java.io.FileOutputStream.open(FileOutputStream.java:270)
	at java.io.FileOutputStream.<init>(FileOutputStream.java:213)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:140)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:159)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:306)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWr